In [2]:
# !pip install fasttext
import fasttext.util
fasttext.util.download_model('ar', if_exists='ignore')  
ft = fasttext.load_model('cc.ar.300.bin')

In [3]:
print(ft.get_dimension())
fasttext.util.reduce_model(ft, 100)
print(ft.get_dimension())


300
100


In [4]:
print(ft.get_word_vector('الشوق شوق')) 
print(ft.get_word_vector('الشوق شوق').shape)

[-0.01018494 -0.08594008 -0.03731866 -0.00024127  0.04777642 -0.00496229
  0.06186691  0.02239722 -0.0280445   0.04849716 -0.00795461 -0.02634357
  0.01323329 -0.02175279  0.03655794 -0.01601862 -0.01084875  0.01393491
 -0.03315404  0.0335445   0.01913922 -0.02036594 -0.00192303 -0.00421568
  0.01183395  0.01175373 -0.07322915  0.08777917  0.00460946 -0.05397879
 -0.04420993 -0.01738087 -0.01531424 -0.02648391  0.05191071 -0.06218194
  0.03248445 -0.03084662  0.02021079 -0.03369771 -0.01912928  0.00776838
 -0.02871501  0.00045804  0.01323607  0.01341687  0.03769046 -0.00100606
  0.03073437 -0.02756213  0.02987276  0.00607357 -0.0034104   0.00598877
 -0.04017798 -0.02268266  0.04346691 -0.00163789  0.03622608  0.00370311
 -0.03838527  0.01520885  0.00647131 -0.02734641  0.01213152  0.03940092
  0.04791509 -0.01371275 -0.00985732 -0.00985206  0.04072648 -0.01279194
  0.02949059  0.0199767   0.01000008  0.0091176  -0.03566917  0.03153419
 -0.05023986 -0.02361483  0.04186894  0.01141178 -0

In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from scipy.stats import skew
import matplotlib.pyplot as plt
import numpy as np

In [7]:
df = pd.read_csv('./Dataset/cleaned_train.csv')
print(df.shape)
df.head()


(6988, 3)


,text,category,stance
0,بيل غيتس تلقي لقاح كوفيد19 من غير تصوير ابر و ...,celebrity,1
1,وزير صح حد يوم تحديد هل بمؤتمروا صحفي كان ما ع...,info_news,1
2,قول رح يكونو اد مسؤولي ب لبنان ما وصل لقاح ؟ ا...,info_news,1
3,تركيا . . وزير صح فخر دين قوجة تلقي اول جرع من...,celebrity,1
4,وئام وهاب شتم دول خليجي في كل طل اعلامي تسافه ...,personal,0


In [23]:
max_length = -1
for tweet in df['text']:
  tweet_arr = tweet.split(' ')
  max_length = max(max_length, len(tweet_arr))

print(max_length)

137


In [53]:
embedded_text = np.zeros((len(df['text']),137*100))
print(embedded_text.shape)
for i,tweet in enumerate(df['text']):
  sentence_embedding = np.array([[]])
  for word in tweet.split(" "):
    sentence_embedding = np.append(sentence_embedding, ft.get_word_vector(word))
  sentence_embedding.resize(137*100)
  print(sentence_embedding.shape)
  embedded_text[i] = sentence_embedding

(6988, 13700)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(13700,)
(1370

In [54]:
type(df['stance'])
print(embedded_text.min())

-1.3525155782699585


In [69]:
x_train, x_test, y_train, y_test = train_test_split(embedded_text, df['stance'], test_size=0.3, random_state=42,stratify=df['stance'])
# print(x_train.shape)
# print(x_test.shape)
x_train = PCA(n_components=100).fit_transform(x_train)
PCA_test = PCA(n_components=100).fit(x_test)
x_test = PCA_test.transform(x_test)
# print(x_train.shape)
# print(x_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

 1    3876
 0     708
-1     307
Name: stance, dtype: int64
 1    1662
 0     304
-1     131
Name: stance, dtype: int64


In [71]:
# clf = Pipeline([
#     ('vectorizer_tfidf', TfidfVectorizer(ngram_range=(1,1), max_df=0.5)),
#     ('ٌRandom Forest', RandomForestClassifier(n_estimators=100, random_state=2002))
# ])
# clf = RandomForestClassifier(n_estimators=100, random_state=2002)
clf = svm.SVC(kernel='linear', C=1.0, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       131
           0       0.00      0.00      0.00       304
           1       0.79      1.00      0.88      1662

    accuracy                           0.79      2097
   macro avg       0.26      0.33      0.29      2097
weighted avg       0.63      0.79      0.70      2097



c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
